# Demonstration the use of API keys with Dask

In [1]:
!pip install -q dask-gateway

In [2]:
from dask_gateway.auth import GatewayAuth
from getpass import getpass
import requests
import jwt
from jwt import PyJWKClient
from datetime import datetime

class DEDL_APIKey_Auth(GatewayAuth):
    
    def __init__(self, apikeyid):
        self.apikeyid = apikeyid
        self.token_url = "https://identity.data.destination-earth.eu/auth/realms/dedl/protocol/openid-connect/token"
        self.cert_url = "https://identity.data.destination-earth.eu/auth/realms/dedl/protocol/openid-connect/certs"
        self.token = self.get_token()
        self.access_token_decoded = self.decode_access_token()
    
    def get_token(self):
        payload = {
            "grant_type": "client_credentials",
            "client_id": self.apikeyid,
            "client_secret": getpass(prompt="Your API Key:")
        }
        return requests.post(self.token_url, data=payload).json()
    
    def decode_access_token(self):
        jwks_client = PyJWKClient(self.cert_url)
        signing_key = jwks_client.get_signing_key_from_jwt(self.token["access_token"])
        return jwt.decode(self.token["access_token"], signing_key.key, audience="account", algorithms=["RS256"])
    
    def token_expired(self):
        if datetime.now() > datetime.fromtimestamp(self.access_token_decoded["exp"]):
            return True
        else:
            return False
    
    def refresh_token_exchange(self):
        payload = {
            "grant_type": "refresh_token",
            "client_id": "dedl-stack-public-client",
            "refresh_token": self.token["refresh_token"],
        }
        return requests.post(self.token_url, data=payload).json()
    
    def refresh(self):
        self.token = self.refresh_token_exchange()
        self.access_token_decoded = self.decode_access_token()
        
        
    def pre_request(self, resp):
        if self.token_expired():
            self.refresh()
        headers = {"Authorization": "Bearer " + self.token["access_token"]}
        return headers, None

### Provide your API Key for authentication

In [3]:
mykey=input()
authenticator = DEDL_APIKey_Auth(apikeyid=mykey)

 stack-internal
Your API Key: ········


## Dask Gateway - Staging Central

In [4]:
from dask_gateway import Gateway
from distributed import Client

dgw_central = Gateway(address="http://217.71.193.55.nip.io",
              proxy_address="tcp://217.71.193.55:80",
              auth=authenticator)

In [5]:
cluster_central = dgw_central.new_cluster()

In [6]:
cluster_central

In [7]:
client = cluster_central.get_client()

/opt/conda/lib/python3.10/site-packages/distributed/client.py:1388: VersionMismatchWarning: Mismatched versions found

+-------------+-----------------+----------------+----------------+
| Package     | Client          | Scheduler      | Workers        |
+-------------+-----------------+----------------+----------------+
| dask        | 2023.5.0        | 2023.8.0       | 2023.8.0       |
| distributed | 2023.5.0        | 2023.8.0       | 2023.8.0       |
| numpy       | 1.23.5          | 1.25.2         | 1.25.2         |
| pandas      | 2.0.1           | 2.0.3          | 2.0.3          |
| python      | 3.10.11.final.0 | 3.11.4.final.0 | 3.11.4.final.0 |
| tornado     | 6.3             | 6.3.2          | 6.3.2          |
+-------------+-----------------+----------------+----------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [8]:
cluster_central.shutdown()